<a href="https://colab.research.google.com/github/ArpanSM/LLM-Notebooks/blob/main/Gemma%202B%20CPP%20GGUF%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub scikit-build-core==0.9.0

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=61" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.62 --force-reinstall --upgrade --no-cache-dir --verbose --no-build-isolation

In [ ]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

In [ ]:
!huggingface-cli login --token "hf_xxxx"

In [ ]:
model_name = "google/gemma-2b-it"
model_file = "gemma-2b-it.gguf"
model_path = hf_hub_download(model_name,
                             filename=model_file,
                             local_dir='/content',
                             token=HF_TOKEN)
print("My model path: ", model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


gemma-2b-it.gguf:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

My model path:  /content/gemma-2b-it.gguf


In [ ]:
model_path = '/content/gemma-2b-it.gguf'

In [ ]:
from llama_cpp import Llama
llm = Llama(model_path=model_path, n_gpu_layers=-1, n_ctx =8192)

llama_model_loader: loaded meta data with 19 key-value pairs and 164 tensors from /content/gemma-2b-it.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_count_kv u32        

In [ ]:
response = llm("State a quote from famous person and explain the meaning.", max_tokens=1000, temperature = 0)


llama_print_timings:        load time =     233.09 ms
llama_print_timings:      sample time =     285.45 ms /    70 runs   (    4.08 ms per token,   245.22 tokens per second)
llama_print_timings: prompt eval time =     232.95 ms /    12 tokens (   19.41 ms per token,    51.51 tokens per second)
llama_print_timings:        eval time =    2893.77 ms /    69 runs   (   41.94 ms per token,    23.84 tokens per second)
llama_print_timings:       total time =    5029.23 ms /    81 tokens


In [ ]:
print(response['choices'][0]['text'].strip())

"The only true wisdom is in knowing you know nothing." - Socrates

**Meaning:** This quote means that the greatest way to learn is to accept that we don't know everything and that continuous growth and exploration are essential for personal development. It encourages us to embrace a lifelong learning mindset and to be open to new experiences and perspectives.


In [ ]:
%%timeit
response = llm("State a quote from famous person and explain the meaning.", max_tokens=1000, temperature = 0)
# 4.61 s ± 179 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2035.84 ms
llama_print_timings:      sample time =     307.69 ms /    70 runs   (    4.40 ms per token,   227.50 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2923.37 ms /    70 runs   (   41.76 ms per token,    23.94 tokens per second)
llama_print_timings:       total time =    5051.07 ms /    71 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    2035.84 ms
llama_print_timings:      sample time =     271.12 ms /    70 runs   (    3.87 ms per token,   258.19 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2895.97 ms /    70 runs   (   41.37 ms per token,    24.17 tokens per second)
llama_print_timings:       to

4.61 s ± 179 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def llama_cpp_predict(prompt):
  response = llm(
    prompt,
    max_tokens=1000,
    temperature = 0.1
  )
  return response['choices'][0]['text'].strip()

def format_prompt(prompt, variable_dict):
    for k,v in variable_dict.items():
        prompt = prompt.replace('{'+k+'}', v)
    return prompt.strip()

In [ ]:
with open('default/classes/AccountFeatureHelper.cls', 'r') as f:
    code_file = f.read()

__code_description_prompt = """
Extract the following information from the given Java code and return it in natural language format:

1. Class Descriptions:
   - Class Name: Identify the purpose of the class.
   - Class Role: Describe the role of the class in the overall system.
   - Inheritance: Note any inheritance relationships (superclasses and interfaces implemented).

2. Method Descriptions:
   - Method Name: State what each method does.
   - Parameters: List and describe the parameters each method accepts.
   - Return Type: Describe what each method returns.
   - Functionality: Explain the main functionality of the method.
   - Exceptions: Mention any exceptions that the method might throw and under what circumstances.

3. Attributes/Fields:
   - Field Name: Describe the purpose of each field.
   - Type: Note the data type of each field.
   - Usage: Explain how and where the field is used within the class.

4. Relationships:
   - Interactions: Describe how the class and methods interact with each other.
   - Dependencies: Note any dependencies on other classes or external libraries.

5. Logic Flow:
   - Control Structures: Describe the use of control structures like loops and conditionals.
   - Key Algorithms: Summarize any important algorithms used in the code.

Java Code:
{java_code}

Description:
"""

cd_prompt = format_prompt(__code_description_prompt, {'java_code': code_file})
response = llama_cpp_predict(cd_prompt)
print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     233.09 ms
llama_print_timings:      sample time =    1850.92 ms /   468 runs   (    3.95 ms per token,   252.85 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   20942.65 ms /   468 runs   (   44.75 ms per token,    22.35 tokens per second)
llama_print_timings:       total time =   33978.72 ms /   469 tokens


This Java code defines a class called `AccountFeatureHelper` that provides methods and fields for handling data related to accounts and features. 

The class has the following methods:

* `getRecordsData`: This method takes a JSON string as input and returns a RecordResponse object containing records filtered based on the provided parameters.
* `getOptions`: This method takes a list of product IDs and field names as input and returns a List of ProductOptionResponse objects, each representing an option for a specific product feature.
* `getChildRelationShipName`: This method takes two object names as input and returns the name of the child relationship between them.

The class also has the following fields:

* `fieldnames`: A list of field names to be retrieved from the database.
* `objectType`: The type of the object being accessed.
* `filters`: A list of filters to apply to the query.
* `staticFilters`: A list of static filters to apply to the query.
* `primarySortField` and `primaryS